## Prevendo Séries Temporais com LSTMs

Dataset Passageiros Aéreos Internacionais (passaporte aéreo).  Os dados contêm o número total de passageiros todos os meses, de janeiro de 1949 a dezembro de 1960. Os números no conjunto de dados referem-se à quantidade em milhares.  Este conjunto de dados foi originalmente usado por Box e Jenkins em seu trabalho em 1976. Ele foi coletado como parte da TimeSeries Dataset Library (TSDL) junto com vários outros conjuntos de dados de séries temporais pelo professor Rob Hyndman na Monash University, Austrália. 

O objetivo nosso é prever quanto de passageiros será em um instante futuro de tempo, para isso usaremos o metodo de janela (window) e o tamanho da janela é um parâmetro que pode ser configurado para cada problema.

In [ ]:
# Imports
import math
import numpy
import pandas
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# Random seed 
numpy.random.seed(7)

In [ ]:
# Carrega o dataset
dataframe = pandas.read_csv('dataset/passageiros.csv', usecols = [1], engine = 'python', skipfooter = 3)
dataset = dataframe.values
dataset = dataset.astype('float32')

In [ ]:
dataset.head()

In [ ]:
# Plot dos dados
passanger_values = dataset.iloc[:,1].values

fig,ax = plt.subplots(figsize=(10,7))
plt.plot(passanger_values,color="green")
plt.xlabel("Time")
plt.ylabel("Passanger Count (Thousand)")
plt.show()

In [ ]:
# Normaliza os dados
scaler = MinMaxScaler(feature_range = (0, 1))
dataset = scaler.fit_transform(dataset)

In [ ]:
# Converte array em matriz
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)


# Split em treino e teste
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [ ]:
# Reshape em X=t e Y=t+1
look_back = 13
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# Reshape do input para: [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
# Cria o modelo LSTM
model = Sequential()
model.add(LSTM(4, input_shape = (1, look_back)))
model.add(Dense(1))


model.compile(loss = 'mean_squared_error', optimizer = 'adam')


model.fit(trainX, trainY, epochs = 200, batch_size = 1, verbose = 2)

In [ ]:
# Previsões
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# Inverte as previsões por conta da normalização
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [ ]:
# Calcula o RMSE
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Score em Treino: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Score em Teste: %.2f RMSE' % (testScore))

In [ ]:
# Ajusta as previsões em treino para o plot
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

# Ajusta as previsões em teste para o plot
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

In [ ]:
# Plot do baseline e previsões
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()